In [4]:
# Import the pandas package, then use the "read_csv" function to read
# the labeled training data
import pandas as pd
import re
from bs4 import BeautifulSoup
nltk.download()

import nltk
from nltk.corpus import stopwords



showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [7]:
train = pd.read_csv("/Users/electromax/Data/jupyter/diploma/github_gold.csv", delimiter=";")

In [9]:
train.head()

,ID,Polarity,Text
0,4063186,neutral,No. I still see the wrong twins. * https://gi...
1,3894703,neutral,"Reverted."""
2,1971084,neutral,You can leave a queue while in queue ? (before...
3,1827828,positive,"Didn't look at SpellTargetRestrictions XD"""
4,232603,neutral,Not sure about what kind of line lengths the p...


In [49]:
def review_to_words( raw_review ):
    review_text = BeautifulSoup(raw_review).get_text() 
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  

    meaningful_words = [w for w in words if not w in stops]   

    return( " ".join( meaningful_words ))   

In [50]:
num_reviews = train["review"].size

clean_train_reviews = [review_to_words(review) for review in train['review']]

In [51]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   # already preprocessed
                             max_features = 10000) 

train_data_features = vectorizer.fit_transform(clean_train_reviews)

train_data_features = train_data_features.toarray()
train_data_features.shape

Creating the bag of words...



(25000, 10000)

In [52]:
import sklearn

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    train_data_features,
    train["sentiment"], 
    test_size=0.2,
    random_state=241
)

import sklearn.metrics
import sklearn.model_selection

def get_classifier_score(classifier):
    predictions = classifier.predict(X_test)
    score = sklearn.metrics.accuracy_score(y_test, predictions)
    return score

split_generator = sklearn.model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
def cross_validation_mean_score(clf):

    score_arr = sklearn.model_selection.cross_val_score(
        estimator=clf,
        X=train_data_features,
        y=train["sentiment"],
        cv=split_generator,
        scoring='neg_mean_squared_error'
    )
    score = score_arr.mean()
    return score

In [53]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 100) 

forest = forest.fit( X_train, y_train )

## Scoring

In [54]:
print(get_classifier_score(forest))

0.8422


In [55]:
# Read the test data
test = pd.read_csv("./input/testData.tsv", header=0, delimiter="\t", quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print(test.shape)

clean_test_reviews = [review_to_words(review) for review in test['review']]

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "./Bag_of_Words_model_submission.csv", index=False, quoting=3 )

(25000, 2)


In [56]:
print(forest.score(train_data_features, train["sentiment"]))

0.96844


In [62]:
positive_review = vectorizer.transform([review_to_words("this film was amazing! i couldnt stop watching")])
negative_review = vectorizer.transform([review_to_words("this film was horrible! i stopped watching immediately")])

print(forest.predict(negative_review))
print(forest.predict(positive_review))

[0]
[1]


## Score
score: 0.84564